In [211]:
import os
import csv
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [212]:
data_folder = '../dataset/'

paths, coolings, heatings = [], [], []
with open('/Users/alex/Downloads/all.csv', 'r') as reader:
    for line in list(reader)[1:]:
        sample = line.split(';')[1:]
        paths.append('/'.join(sample[0].split('/')[2:]))
        coolings.append(int(sample[1]))
        heatings.append(int(sample[2][:-2]))

heatings = np.array(heatings)
coolings = np.array(coolings)

data_folder = '../dataset/'

paths_all = []
with open('/Users/alex/Downloads/all.csv', 'r') as reader:
    for line in list(reader)[1:]:
        sample = line.split(';')[1:]
        paths_all.append('/'.join(sample[0].split('/')[2:]))

num_samples = len(paths_all)
print(num_samples)

1468


In [213]:
room_types = ['living_room', 'kitchen', 'bedroom',
    'bathroom', 'restroom', 'washing_room',
    'office', 'closet', 'balcony',
    'corridor', 'dining_room','laundry_room',
    'PS']


In [214]:
def extract_file(file) :
    area = 0
    door_length, wall_length = 0, 0
    nb_door, nb_wall = 0,0
    max_number = 0
    with open(file,'r') as reader:
            lines_wall = []
            lines_door = []
            for i, line in enumerate(reader): 
                line_main = line.split()
                line = [int(float(x)) for x in line_main[:4]]
                for coord in line :
                    if coord > max_number :
                        max_number = coord
                if 'door' in line_main :
                    door_length += (abs(line[0] - line[2]) + abs(line[1]-line[3]))
                    nb_door+=1
                elif 'wall' in line_main :
                    wall_length += abs(line[0] - line[2])+ abs(line[1]-line[3])
                    nb_wall+=1
                elif line_main[4] in room_types:
                    area += (abs(line[0] - line[2]) * abs(line[1]-line[3]))

    return max_number, door_length, wall_length, area, nb_door, nb_wall

In [220]:
with open('energy_main__.csv', 'w') as f :
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['max_number', 'door_length', 'wall_length', 'area','nb_door','nb_wall' ,'cooling', 'heating'])
    for index, path in enumerate(paths) :
        file = os.listdir(os.path.join(data_folder, path))[0]
        max_number, door_length, wall_length, area, nb_door, nb_wall = extract_file(os.path.join(data_folder, path, file))
        writer.writerow([max_number, door_length, wall_length, area, nb_door, nb_wall, coolings[index], heatings[index]])
#table = pd.read_csv('energy_main.csv', delimiter = ";")
#print(table)

In [233]:
def process(path_data, training=True, MinMax=None):

    x_columns = ['max_number', 'door_length', 'wall_length', 'area','nb_door','nb_wall']
    #x_columns = ['volume', 'nb_people', 'wall_area', 'door_area','roof_area', 'space']
    y1_column = ['cooling']
    y2_column = ['heating']
    
    data = pd.read_csv(path_data, delimiter =";")
    
    print(data)
    X = data[x_columns]
    Y1 = data[y1_column]
    Y2 = data[y2_column]
    if training :
        """ Splitting """
        X_train_div, X_test_div, y1_train, y1_test = train_test_split(
            X, Y1, random_state=5, test_size=0.2)
        X_train_div, X_test_div, y2_train, y2_test = train_test_split(
            X, Y2, random_state=5, test_size=0.2)

        """ Scaling """
        MinMax = MinMaxScaler(feature_range=(0, 1))
        X_train_div = MinMax.fit_transform(X_train_div)
        X_test_div = MinMax.transform(X_test_div)
        #return X_train_div, X_test_div, y1_train.to_numpy(), y1_test.to_numpy(), y2_train.to_numpy(), y2_test.to_numpy(), MinMax
        return X_train_div, X_test_div, y1_train, y1_test, y2_train, y2_test, MinMax, data

    """ Scaling with previous scaler - If test """

    X = MinMax.transform(X)
    return X, data


In [234]:
path_data = 'energy_main.csv'
X_train_div, X_test_div, y1_train, y1_test, y2_train, y2_test, minmax, data = process(
    path_data, training=True)

      max_number  door_length  wall_length   area  nb_door  nb_wall  cooling  \
0            533          185         1036   1000        3        6    11254   
1            475           42          392   1000        1        4    13620   
2            632            0         2076   1000        0        6    14669   
3            476           74          396   1000        1        4    14640   
4            474          289         2248   1000        8       36    16897   
...          ...          ...          ...    ...      ...      ...      ...   
1195         347          808         3091  19000       19       64  1605180   
1196         491          677         3118  19000       18       65  1665228   
1197         372          739         3039  19000       17       57  1665543   
1198         620          769         4357  20800       16       65  1689439   
1199         518          879         3729  22000       26       68  1772799   

      heating  
0       11304  
1      

In [235]:
#Random Forest
param_grid = {'max_features': ['auto', 'log2'],
              'max_depth': [100]}  # 10,15,20,30,50,60, also possible change n_estimators
# model
model = RandomForestRegressor(random_state=5, n_estimators=2000, n_jobs=-1)



In [236]:
#XGB
params = {'max_depth':10 , 'min_samples_split': 2,
          'learning_rate': 0.0001, 'loss': 'ls'}
model = GradientBoostingRegressor(n_estimators = 100)

In [192]:
""" Cooling """
grid_search_rf = MultiOutputRegressor(GridSearchCV(
    model, param_grid, cv=5, return_train_score=True, verbose=2))
grid_search_rf.fit(X_train_div, y1_train)

print('The Train R2 score for cooling load is', r2_score(
    y1_train, grid_search_rf.predict(X_train_div)))
print('The Test R2 score for cooling load is', r2_score(
    y1_test, grid_search_rf.predict(X_test_div)))

""" Heating """
grid_search_rf2 = MultiOutputRegressor(GridSearchCV(
    model, param_grid, cv=5, return_train_score=True, verbose=2))
grid_search_rf2.fit(X_train_div, y2_train)

print('The Train R2 score for heating load is', r2_score(
    y2_train, grid_search_rf2.predict(X_train_div)))
print('The Test R2 score for heating load is', r2_score(
    y2_test, grid_search_rf2.predict(X_test_div)))


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] max_depth=100, max_features=auto ................................
[CV] ................. max_depth=100, max_features=auto, total=   3.8s
[CV] max_depth=100, max_features=auto ................................
[CV] ................. max_depth=100, max_features=auto, total=   2.1s
[CV] max_depth=100, max_features=auto ................................
[CV] ................. max_depth=100, max_features=auto, total=   1.9s
[CV] max_depth=100, max_features=auto ................................
[CV] ................. max_depth=100, max_features=auto, total=   1.8s
[CV] max_depth=100, max_features=auto ................................
[CV] ................. max_depth=100, max_features=auto, total=   1.8s
[CV] max_depth=100, max_features=log2 ................................
[CV] ................. max_depth=100, max_features=log2, total=   1.8s
[CV] max_depth=100, max_features=log2 ................................
[CV] ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   22.7s finished


In [223]:
### TEST ###
PATH = '../dataset/vectors/'

paths_test, coolings_test, heatings_test = [], [], []

print('Test data loading...')

for path_1 in os.listdir(PATH):
    if path_1 == '_DS_Store' or path_1 == '.DS_Store':
        continue
    PATH_1 = os.path.join(PATH, path_1)
    for path_2 in os.listdir(PATH_1):
        if path_2 == '.DS_Store':
            continue
        PATH_2 = os.path.join(PATH_1, path_2)
        for path_3 in os.listdir(PATH_2):
            if path_3 == '.DS_Store':
                continue
            PATH_3 = os.path.join(PATH_2, path_3)
            txt_path = os.listdir(PATH_3)[0]
            txt_path = os.path.join(PATH_3, txt_path)
            paths_test.append(txt_path)
print(len(paths_test))

Test data loading...
101046


In [263]:
with open('energy_all.csv', 'w') as f :
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['paths','max_number', 'door_length', 'wall_length', 'area','nb_door','nb_wall' ,'cooling', 'heating'])
    for index, path in enumerate(paths_test) :
        max_number, door_length, wall_length, area, nb_door, nb_wall = extract_file(path)
        writer.writerow([path[19:-9],max_number, door_length, wall_length, area, nb_door, nb_wall])

In [264]:
path_data_prediction = 'energy_all.csv'
# MinMax to scale prediction data
X, data_test = process(path_data_prediction, training=False, MinMax=minmax)

                                     paths  max_number  door_length  \
0       0c/61/6407788e3dc8ab96802754f29454         454          865   
1       0c/61/9a70ffd39eb8b5ddea8251ae64d2         616          487   
2       0c/61/82be7dacf24447ad01ef9ef20ee4        1869         1850   
3       0c/61/c3df18a509d7d4c440f9530b81d3         437          418   
4       0c/61/5eb69b10672997d4131d2f5ab622        1092         1158   
...                                    ...         ...          ...   
101041  0a/06/5a342c44a5483c5f7e5989478212         329          181   
101042  0a/06/4e5a5332e0a19729ae0034d3420d         785          760   
101043  0a/06/dd32b00e8d51f43123f64ea81a98        1885          807   
101044  0a/06/001fc1fcde5accb860e2d59389f9         476          519   
101045  0a/06/3dc3c277b926eeb841dc3b5c9510         417          476   

        wall_length   area  nb_door  nb_wall  cooling  heating  
0              3338  18000       22       59      NaN      NaN  
1              24

In [238]:
cooling_pred = grid_search_rf.predict(X)
prediction_cooling = pd.DataFrame(
    cooling_pred, columns=['Cooling_Load'])

heating_pred = grid_search_rf2.predict(X)
prediction_heating = pd.DataFrame(
    heating_pred, columns=['Heating_Load'])

In [262]:
path_dfs = data_test['paths']
full = pd.concat([path_dfs, prediction_cooling.apply(np.int64), prediction_heating.apply(np.int64)], axis = 1).to_csv('all.csv')